In [6]:
DATA_BUCKET_URI = 'gs://vtx-datasets-public/Task01_BrainTumour/'

In [8]:
DATA_BUCKET_URI.split('/')[2]

'vtx-datasets-public'

In [42]:


def preprocess():
    import os
    import shutil
    from google.cloud import storage

    bucket = DATA_BUCKET_URI.split('/')[2]
    folder = DATA_BUCKET_URI.split('/')[3]

    storage_client = storage.Client(project='jchavezar-demo')
    bucket=storage_client.get_bucket(bucket)
    blobs=bucket.list_blobs(prefix=folder)

    if os.path.exists(f'/tmp/{folder}'):
        shutil.rmtree(f'/tmp/{folder}')
    os.mkdir(f'/tmp/{folder}')
    for subfolder in ['imagesTr', 'imagesTs', 'labelsTr']:
        os.mkdir(f'/tmp/{folder}/{subfolder}')
    for blob in blobs:
        print(blob.name)
        destination_uri = f'/tmp/{blob.name}'
        blob.download_to_filename(destination_uri)

In [49]:
%%writefile Dockerfile
FROM --platform=amd64 ubuntu:20.04
ENV GCSFUSE_REPO gcsfuse-stretch

USER root

# Add google repositories for gcsfuse and google cloud sdk
RUN apt-get update -y && apt-get install -y --no-install-recommends apt-transport-https ca-certificates curl gnupg
RUN echo "deb http://packages.cloud.google.com/apt $GCSFUSE_REPO main" | tee /etc/apt/sources.list.d/gcsfuse.list
RUN echo "deb https://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
RUN curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -

# Install gcsfuse and google cloud sdk
RUN apt-get update -y  && apt-get install -y gcsfuse google-cloud-sdk \
    && apt-get autoremove -y \
    && apt-get clean -y \
    && rm -rf /var/lib/apt/lists/* /tmp/* /var/tmp/*
ENTRYPOINT ["gcsfuse"]

Overwriting Dockerfile


In [50]:
!gcloud builds submit -t gcr.io/jchavezar-demo/alpine-gcs:v1 .

Creating temporary tarball archive of 2 file(s) totalling 4.4 KiB before compression.
Uploading tarball of [.] to [gs://jchavezar-demo_cloudbuild/source/1680109438.441807-cdb0579cd40e4bb7bc282e1e545e8dea.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/6ab8bcab-6570-464c-a18c-515cf93475b9].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/6ab8bcab-6570-464c-a18c-515cf93475b9?project=569083142710 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "6ab8bcab-6570-464c-a18c-515cf93475b9"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1680109438.441807-cdb0579cd40e4bb7bc282e1e545e8dea.tgz#1680109438640940
Copying gs://jchavezar-demo_cloudbuild/source/1680109438.441807-cdb0579cd40e4bb7bc282e1e545e8dea.tgz#1680109438640940...
/ [1 files][  1.5 KiB/  1.5 KiB]                                                
Operation completed over 1 objects/1

In [54]:
from kfp.dsl import component

help(component)

Help on function component in module kfp.dsl._component:

component(func)
    Decorator for component functions that returns a ContainerOp.
    
    This is useful to enable type checking in the DSL compiler.
    
    Example:
      ::
    
        @dsl.component
        def foobar(model: TFModel(), step: MLStep()):
          return dsl.ContainerOp()



In [58]:
from kfp.dsl import component


@component(base_image="python:latest")
def test():
    import np
    print(np.array([[1,2],[3,4]]))

TypeError: component() got an unexpected keyword argument 'base_image'

In [62]:
!pip install kfp -pre


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -p


In [63]:
!pip install kfp --pre
